## Playing with the mask concept
Basically only tag the first transformer token of a spacy token

In [33]:
from allennlp.data import Instance
from allennlp.data.fields import TextField
from allennlp.data import Token
from allennlp.data.token_indexers import PretrainedTransformerIndexer
from allennlp.data import Batch
from allennlp.data import Vocabulary

In [3]:
df_valid = pd.read_json("valid_v1.json", orient="records", lines=True)

In [16]:
model_name="dccuchile/bert-base-spanish-wwm-uncased"
tokenizer = AutoTokenizer.from_pretrained(model_name)

In [20]:
input_ids = tokenizer(df_valid.tokens[0], is_split_into_words=True, return_offsets_mapping=True)
allennlp_tokens = list(map(Token, tokenizer.convert_ids_to_tokens(input_ids["input_ids"])))

In [28]:
indexer = PretrainedTransformerIndexer(model_name=model_name)
text_field = TextField(allennlp_tokens, token_indexers={"transformers": indexer})
instance = Instance({"tokens": text_field})

In [48]:
vocab = Vocabulary.from_instances([instance])
batch = Batch([instance])
batch.index_instances(vocab)
token_ids = batch.as_tensor_dict()["tokens"]["transformers"]
indices = indexer.tokens_to_indices(allennlp_tokens, vocab)

In [49]:
def check_offsets(df, model_name="dccuchile/bert-base-spanish-wwm-uncased") -> Tuple[List, List]:
    indexer = PretrainedTransformerIndexer(model_name=model_name)
    vocab = Vocabulary()
    tokenizer = AutoTokenizer.from_pretrained(model_name) 
    
    for row in tqdm(df.itertuples(), total=len(df)):
        indices = tokenizer(row.tokens, return_offsets_mapping=True, is_split_into_words=True, return_special_tokens_mask=True)
        print
        token_groups = []
        for input_id, offset in zip(indices["input_ids"][1:-1], indices["offset_mapping"][1:-1]):
            if offset[0] == 0:
                token_groups.append([tokenizer.convert_ids_to_tokens(input_id)])
            else:
                token_groups[-1].append(tokenizer.convert_ids_to_tokens(input_id))
        print(list(zip(row.tokens, token_groups)))
        tokens_str = tokenizer.convert_ids_to_tokens(indices["input_ids"])
        tokens = [Token(tok) for tok in tokens_str]
        token_indexes = indexer.tokens_to_indices(tokens, vocabulary=vocab)
        token_ids = token_indexes["token_ids"]
                    
    return destroyed_tokens, destroyed_tags, offs

In [50]:
check_offsets(df_valid[:2])

[('COMUNICADO', ['comunicado']), ('POR', ['por']), ('CORONAVIRUS', ['corona', '##vir', '##us']), ('\n', ['el']), ('El', ['presidente']), ('Presidente', ['ruso']), ('Ruso', ['vladimir']), ('Vladimir', ['pu', '##tin']), ('Putin', ['ha']), ('ha', ['dicho']), ('dicho', [':']), (':', ['"']), ('"', ['los']), ('los', ['ciudadanos']), ('ciudadanos', ['rusos']), ('rusos', ['tienen']), ('tienen', ['dos']), ('dos', ['opciones']), ('opciones', [',']), (',', ['se']), ('se', ['quedan']), ('quedan', ['en']), ('en', ['su']), ('su', ['casa']), ('casa', ['por']), ('por', ['15']), ('15', ['dias']), ('días', ['o']), ('o', ['van']), ('van', ['a']), ('a', ['prision']), ('prisión', ['por']), ('por', ['5']), ('5', ['anos']), ('años', ['"']), ('"', ['fin']), ('FIN', ['del']), ('DEL', ['comunicado']), ('COMUNICADO', ['.']), ('.', ['[UNK]']), (' ', ['nom', '##e', '##que', '##do', '##en', '##cas', '##a']), ('#', ['h', '##tt', '##ps', ':', '[UNK]', '[UNK]', 't', '.', 'co', '[UNK]', 'z', '##ml', '##w', '##un', '##v

NameError: name 'destroyed_tokens' is not defined

In [30]:
from biome.text import Dataset

In [32]:
ds = Dataset.from_json("valid_v1.json")

Using custom data configuration default
Reusing dataset json (/home/david/.cache/huggingface/datasets/json/default-b2dd006bd782a885/0.0.0/70d89ed4db1394f028c651589fcab6d6b28dddcabbe39d3b21b4d41f9a708514)


In [39]:
indices = tokenizer(ds[0]["tokens"], return_offsets_mapping=True, is_split_into_words=True, return_special_tokens_mask=True)

In [42]:
i = 0
for off in indices["offset_mapping"]:
    if off[0] == 0:
        i += 1
i

44

In [43]:
len(ds[0]["tags_bio"])

44

In [23]:
from allennlp.training.metrics import SpanBasedF1Measure
from allennlp.data.vocabulary import Vocabulary
import torch

In [24]:
f1 = SpanBasedF1Measure(Vocabulary())

In [ ]:
pred = [["B-TEST", "I-TEST", "O"], ...]
gold = ["B-TEST", "O", "O"]
vocab = {"B-TEST": 0, "I-TEST": 1, "O": 2}

vocab = {"B-8474693", "I-...", ...........}

In [22]:
predictions = torch.tensor([[[1, 0, 0], [0, 1, 0], [0, 0, 1]], [...], [...]]) 
gold = torch.tensor([[[1, 0, 0], [0, 0, 1], [0, 0, 1]]])

len(predictions.size()) == 3
# batchsize, nr of words, nr of labels

Signature:     
f1(
    predictions: torch.Tensor,
    gold_labels: torch.Tensor,
    mask: Union[torch.BoolTensor, NoneType] = None,
    prediction_map: Union[torch.Tensor, NoneType] = None,
)
Type:           SpanBasedF1Measure
String form:    <allennlp.training.metrics.span_based_f1_measure.SpanBasedF1Measure object at 0x7f712113e6d0>
File:           ~/miniconda3/envs/biome/lib/python3.7/site-packages/allennlp/training/metrics/span_based_f1_measure.py
Docstring:     
The Conll SRL metrics are based on exact span matching. This metric
implements span-based precision and recall metrics for a BIO tagging
scheme. It will produce precision, recall and F1 measures per tag, as
well as overall statistics. Note that the implementation of this metric
is not exactly the same as the perl script used to evaluate the CONLL 2005
data - particularly, it does not consider continuations or reference spans
as constituents of the original span. However, it is a close proxy, which
can be helpful for judg

In [25]:
f1(predictions, gold, mask)

Signature:     
f1(
    predictions: torch.Tensor,
    gold_labels: torch.Tensor,
    mask: Union[torch.BoolTensor, NoneType] = None,
    prediction_map: Union[torch.Tensor, NoneType] = None,
)
Type:           SpanBasedF1Measure
String form:    <allennlp.training.metrics.span_based_f1_measure.SpanBasedF1Measure object at 0x7f7121209e90>
File:           ~/miniconda3/envs/biome/lib/python3.7/site-packages/allennlp/training/metrics/span_based_f1_measure.py
Docstring:     
The Conll SRL metrics are based on exact span matching. This metric
implements span-based precision and recall metrics for a BIO tagging
scheme. It will produce precision, recall and F1 measures per tag, as
well as overall statistics. Note that the implementation of this metric
is not exactly the same as the perl script used to evaluate the CONLL 2005
data - particularly, it does not consider continuations or reference spans
as constituents of the original span. However, it is a close proxy, which
can be helpful for judg